In [1]:
# All the necessary and usable imports for the project:

import os
import json
import pathlib
import random
import time
import zipfile
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
data = []

# Driver declaration and initiation:

def driver_return():
    chromeOptions = webdriver.ChromeOptions()
    chromeOptions.add_argument("--window-size=150,500")
    chromeOptions.add_argument("--disable-automation")
    chromeOptions.add_argument("--no-sandbox")
    chromeOptions.add_argument('--profile-directory=Default')
    chromeOptions.add_argument("--start-maximized")
    chromeOptions.add_argument("--lang=en")
    chromeOptions.add_experimental_option("prefs", {
        "download.default_directory": r"C:\Users\hp\Desktop\eais.go.kr\downloads",
    })
    chromeOptions.add_argument("--enable-javascript")
    chromeOptions.add_argument("--enable-cookies")
    chromeOptions.add_argument("--disable-popup-blocking")
    chromeOptions.add_extension(r'C:\Users\hp\Desktop\eais.go.kr\alert_blocker.crx')
    chromeOptions.add_experimental_option("detach", True)
    scriptDirectory = pathlib.Path().absolute()
    chromeOptions.add_argument(f"user-data-dir={scriptDirectory}\\userdata")
    driver = webdriver.Chrome(service=ChromeService(r'C:\Users\hp\Desktop\eais.go.kr\chromedriver.exe'), options=chromeOptions)
    return driver

with open(r'C:\Users\hp\Desktop\eais.go.kr\formatted_addresses.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

driver = driver_return()
driver.maximize_window()

In [3]:
# Automatic log in in to account:

driver.get("https://www.eais.go.kr/")
print("Logging In...")

wait = WebDriverWait(driver, 10)
try:
    wait.until(EC.visibility_of_element_located((By.XPATH, '//button[@class="btnLogin btnLine btnNormal btnLine_blue"]')))
    login_main = driver.find_element(By.XPATH, '//button[@class="btnLogin btnLine btnNormal btnLine_blue"]')
    driver.execute_script("arguments[0].click();", login_main)

    wait.until(EC.visibility_of_element_located((By.XPATH, '//input[@id="membId"]')))
    uname = driver.find_element(By.XPATH, '//input[@id="membId"]')
    uname.send_keys("fockbroke003")
    
    wait.until(EC.visibility_of_element_located((By.XPATH, '//input[@name="pwd"]')))
    pwd = driver.find_element(By.XPATH, '//input[@name="pwd"]')
    pwd.send_keys("589896@#$12")

    login_btn = driver.find_element(By.XPATH, '//button[@class="btnLogin"]')
    driver.execute_script("arguments[0].click();", login_btn)

    wait.until(EC.visibility_of_element_located((By.XPATH, '//input[@id = "searchBox"]')))
    print("Login Successful...")
except:
    print("Already Logged In...")
    pass

Logging In...
Login Successful...


In [5]:
# This part of the code is responsible for all the inputs and clicks on the page to perform step by step process and get to the final pdf:
for item in data:
    # driver.get("https://www.eais.go.kr/moct/bci/aaa02/BCIAAA02L01")
    address = item["Address"]
    building_number = str(item["Building"])
    unit_number = str(item["Unit"])
    
    print("\nGetting insurance page for address: ", address)
    
    # Setting the implicit wait as required to wait for the elements to render, depending on the internet connection and local isp:
    wait = WebDriverWait(driver, 20)
    wait.until(EC.invisibility_of_element_located((By.XPATH, '//span[@class="w2span"]')))
    wait.until(EC.visibility_of_element_located((By.XPATH, '//span[@class="multiselect__placeholder"]')))
    input_click = driver.find_element(By.XPATH, '//span[@class="multiselect__placeholder"]')
    input_click.click()
    
    try:
        wait.until(EC.invisibility_of_element_located((By.XPATH, '//span[@class="w2span"]')))
        wait.until(EC.visibility_of_element_located((By.XPATH, '//input[@class="multiselect__input"]')))
        search_bar = driver.find_element(By.XPATH, '//input[@class="multiselect__input"]')
        search_bar.send_keys(address)
        print("Address entered: ", address)
    except:
        print("Address not entered...")

    wait.until(EC.visibility_of_element_located((By.XPATH, '(//div[@class="option__desc"])[1]')))
    option = driver.find_element(By.XPATH, '(//div[@class="option__desc"])[1]')
    option.click()

    try:
        F_building_number = building_number + "동"
        wait.until(EC.invisibility_of_element_located((By.XPATH, '//span[@class="w2span"]')))
        wait.until(EC.visibility_of_element_located((By.XPATH, f'//span[@class="option__title"][contains(text(), "{F_building_number}")]')))
        building = driver.find_element(By.XPATH, f'//span[@class="option__title"][contains(text(), "{F_building_number}")]')
        building.click()
        print("Building number entered: ", F_building_number)
    except:
        print("Building number not entered...")

    try:
        # F_unit_number = unit_number + "호"
        wait.until(EC.invisibility_of_element_located((By.XPATH, '//span[@class="w2span"]')))
        wait.until(EC.visibility_of_element_located((By.XPATH, f'//span[@class="option__title"][contains(text(), "{unit_number}")]')))
        unit = driver.find_element(By.XPATH, f'//span[@class="option__title"][contains(text(), "{unit_number}")]')
        unit.click()
        print("Unit number entered: ", unit_number)
    except:
        print("Unit number not entered...")

    wait.until(EC.invisibility_of_element_located((By.XPATH, '//span[@class="w2span"]')))
    wait.until(EC.visibility_of_element_located((By.XPATH, '//button[@class="btnSrch"]')))
    search_btn = driver.find_element(By.XPATH, '//button[@class="btnSrch"]')
    search_btn.click()
    print("Viewing Search Results...")
    
    time.sleep(2)
    wait.until(EC.visibility_of_element_located((By.XPATH, '//div[@class="ag-root ag-unselectable ag-layout-normal"][1]')))
    wait.until(EC.invisibility_of_element_located((By.XPATH, '//span[@class="w2span"]')))
    wait.until(EC.visibility_of_element_located((By.XPATH, '(//div[@class="compSelTblBody"][1]//div[@role="gridcell"])[1]')))
    blue_option = driver.find_element(By.XPATH, '(//div[@class="compSelTblBody"][1]//div[@role="gridcell"])[1]')
    blue_option.click()

    wait.until(EC.invisibility_of_element_located((By.XPATH, '//span[@class="w2span"]')))
    wait.until(EC.visibility_of_element_located((By.XPATH, '//button[@title="신청할 민원 추가"]')))
    add_to_cart = driver.find_element(By.XPATH, '//button[@title="신청할 민원 추가"]')
    add_to_cart.click()
    print("Added to cart...")

    wait.until(EC.invisibility_of_element_located((By.XPATH, '//span[@class="w2span"]')))
    wait.until(EC.visibility_of_element_located((By.XPATH, '//button[@class="btnSubmit mt10"]')))
    btn_submit = driver.find_element(By.XPATH, '//button[@class="btnSubmit mt10"]')
    btn_submit.click()
    print("Query Submitted...")

    # Setting an explicit wait because of an alert and an invisible overlay which interrupts the click on the element:
    time.sleep(5)
    wait.until(EC.invisibility_of_element_located((By.XPATH, '//span[@class="w2span"]')))
    wait.until(EC.visibility_of_element_located((By.XPATH, '//button[@class="btnNext btnSolid btnLarge btn_blue"]')))
    apply_btn = driver.find_element(By.XPATH, '//button[@class="btnNext btnSolid btnLarge btn_blue"]')
    apply_btn.click()

    time.sleep(5)
    wait.until(EC.invisibility_of_element_located((By.XPATH, '//span[@class="w2span"]')))
    wait.until(EC.visibility_of_element_located((By.XPATH, '//a[@class="tbsBadge st1"][contains(text(), "테스트인쇄")]')))
    pdf_pop = driver.find_element(By.XPATH, '//a[@class="tbsBadge st1"][contains(text(), "테스트인쇄")]')
    pdf_pop.click()

    print("Fetching pdf...")
    driver.switch_to.window(driver.window_handles[1])
    driver.maximize_window()
    wait.until(EC.invisibility_of_element_located((By.XPATH, '//span[@class="w2span"]')))
    pdf_link = driver.current_url

    wait.until(EC.visibility_of_element_located((By.XPATH, '//button[@title="PDF 저장"]')))
    pdf_download = driver.find_element(By.XPATH, '//button[@title="PDF 저장"]')
    pdf_download.click()
    
    # Finally downloading the pdf file to local storage to the project download directory:
    wait.until(lambda driver: any(filename.endswith('report.pdf') for filename in os.listdir('C:\\Users\\hp\\Desktop\\eais.go.kr\\downloads')))

    # By default the pdf is downloaded with the name of "report" which is then renamed to the "address + building_no + unit_no"
    os.rename(os.path.join('C:\\Users\\hp\\Desktop\\eais.go.kr\\downloads', 'report.pdf'), os.path.join('C:\\Users\\hp\\Desktop\\eais.go.kr\\downloads', f'{address + F_building_number + unit_number}.pdf'))
    print("PDF saved for address: ", address)
    driver.close()
    driver.switch_to.window(driver.window_handles[0])


Getting insurance page for address:  서울특별시 강남구 언주로 332 역삼푸르지오
Address entered:  서울특별시 강남구 언주로 332 역삼푸르지오
Building number entered:  101동
Unit number entered:  101
Viewing Search Results...
Added to cart...
Query Submitted...
Fetching pdf...
PDF saved for address:  서울특별시 강남구 언주로 332 역삼푸르지오

Getting insurance page for address:  서울특별시 강남구 언주로 332 역삼푸르지오


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6CB3652A2+57122]
	(No symbol) [0x00007FF6CB2DEA92]
	(No symbol) [0x00007FF6CB1AE3AB]
	(No symbol) [0x00007FF6CB1E7D3E]
	(No symbol) [0x00007FF6CB1E7E2C]
	(No symbol) [0x00007FF6CB220B67]
	(No symbol) [0x00007FF6CB20701F]
	(No symbol) [0x00007FF6CB21EB82]
	(No symbol) [0x00007FF6CB206DB3]
	(No symbol) [0x00007FF6CB1DD2B1]
	(No symbol) [0x00007FF6CB1DE494]
	GetHandleVerifier [0x00007FF6CB60EF82+2849794]
	GetHandleVerifier [0x00007FF6CB661D24+3189156]
	GetHandleVerifier [0x00007FF6CB65ACAF+3160367]
	GetHandleVerifier [0x00007FF6CB3F6D06+653702]
	(No symbol) [0x00007FF6CB2EA208]
	(No symbol) [0x00007FF6CB2E62C4]
	(No symbol) [0x00007FF6CB2E63F6]
	(No symbol) [0x00007FF6CB2D67A3]
	BaseThreadInitThunk [0x00007FFBD76E26AD+29]
	RtlUserThreadStart [0x00007FFBD890AA68+40]
